In [5]:
%matplotlib widget
from scipy.signal import convolve
from process_cube.dat import read_dat_file, Header
import numpy as np
from tqdm.notebook import tqdm
import numba

def neighbour_mean(x):
    kernel = np.ones((3,3))
    kernel[1,1] =0
    kernel = kernel[...,np.newaxis]
    out = convolve(x, kernel,"same")
    out[0,0]/=3
    out[-1,-1]/=3
    out[-1,0]/=3
    out[0,-1]/=3
    out[0,1:-1]/=5
    out[-1,1:-1]/=5
    out[1:-1,0]/=5
    out[1:-1,-1]/=5
    out[1:-1,1:-1]/=8
    return out

@numba.njit
def estimation(t,phi_inv, data, M):
    est = np.zeros(data.shape[:2])
    for i,row in enumerate(data):
        for j,pixel in enumerate(row):
            est[i,j] = t @ (phi_inv @ (pixel - M[i,j]))
    return est

def covariance_matrix(x):
    covariance = np.zeros((x.shape[-1], x.shape[-1]))
    for i,row in enumerate(x):
        for j,pixel in enumerate(row):
            covariance += pixel.reshape(-1,1) * pixel.reshape(1,-1)
            
    return covariance/x.shape[0]/x.shape[1]

def bins_centers(bins):
    return (bins[:-1] + bins[1:])/2

def get_shared_bins(count, *args):
    low = np.min(args)
    high = np.max(args)
    return np.linspace(low, high, count+1)

def original_algorithm(data, p, t):
    M = neighbour_mean(data)
    data_tag = data + p*t
    phi = covariance_matrix(data-M)
    phi_inv = np.linalg.inv(phi)
    est_no_target = estimation(t,phi_inv, data, M)
    est_with_target = estimation(t,phi_inv, data_tag, M)
    return est_no_target, est_with_target

header = Header.read_header("self_test_rad.hdr")
img = read_dat_file("self_test_rad.img", header).astype(np.float64)
p = 0.02
t = img[60,70,:]
no_target, with_target= original_algorithm(img, p, t)

C:\Users\Shaiel Cohen\AppData\Local\Temp\ipykernel_25820\1841775795.py:29: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'C', False, aligned=True))
  est[i,j] = t @ (phi_inv @ (pixel - M[i,j]))


In [6]:
import json
from tqdm.notebook import tqdm

@numba.njit
def clustered_estimation(t, data, M, clustered_image, clusters_covariance):
    est = np.zeros(data.shape[:2])
    for i,row in enumerate(data):
        for j,pixel in enumerate(row):
            k = clustered_image[i,j]
            est[i,j] = t @ (clusters_covariance[k] @ (pixel - M[i,j]))
    return est

def clustered_algorithm(data, p, t, clustered_image):
    M = neighbour_mean(data)
    data_tag = data + p*t
    clusters_covariance = np.array([np.linalg.inv(covariance_matrix((data-M)[clustered_image == k][np.newaxis])) for k in tqdm(range(len(cluster_names)))])
    est_no_target = clustered_estimation(t, data, M, clustered_image, clusters_covariance)
    est_with_target = clustered_estimation(t, data_tag, M, clustered_image, clusters_covariance)
    return est_no_target, est_with_target

# Read clusters
with open("clusters.json") as f:
    clusters = json.load(f)

clusters_indexes = {tuple(cluster["name"]): cluster["cluster"] for cluster in clusters.values()}
cluster_names = list(clusters_indexes.keys())
clusters_pixel_index = list(clusters_indexes.values())
clustered_image = np.zeros((img.shape[0], img.shape[1]), dtype=int)

for k, cluster_pixel_index in enumerate(clusters_pixel_index):
    for i,j in cluster_pixel_index:
        clustered_image[i,j] = k

clustered_est_no_target, clustered_est_with_target = clustered_algorithm(img, p, t, clustered_image)

  0%|          | 0/7 [00:00<?, ?it/s]

In [7]:

from matplotlib import pyplot as plt
def get_shared_bins(count, *args):
    low = np.min(args)
    high = np.max(args)
    return np.linspace(low, high, count+1)
no_target, with_target= original_algorithm(img, p, t)
bins = get_shared_bins(1000, no_target, with_target)
count_no_target,bins_no_target = np.histogram(no_target.reshape(-1,),bins=bins)
count_with_target,bins_with_target =np.histogram(with_target.reshape(-1,),bins=bins)
fig = plt.figure()
plt.plot(bins_centers(bins_no_target), count_no_target)
plt.plot(bins_centers(bins_with_target), count_with_target)
plt.legend(["no-target", "with-target"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
from scipy.interpolate import pchip

def get_inv_cdf(count, bins):
    cdf = np.cumsum(count)/np.sum(count)
    return 1 - cdf/cdf[-1]

def extend_cdf_range(samples, sample_points, new_sample_points):
    out = np.zeros(len(new_sample_points))
    current_point_index = 1
    current_point = sample_points[current_point_index]
    current_sample = samples[current_point_index]
    for i, sample_point in enumerate(new_sample_points):
        if sample_point >= current_point:
            current_point_index += 1
        if current_point_index >= len(sample_points):
            current_sample = 0
        else: 
            current_sample = samples[current_point_index-1]
        out[i] = current_sample
    return out

inv_cdf_no_target = get_inv_cdf(count_no_target, bins_no_target)
inv_cdf_with_target = get_inv_cdf(count_with_target, bins_no_target)
value_range = np.sort(np.concatenate((bins_no_target,bins_with_target)))
inv_cdf_no_target = extend_cdf_range(inv_cdf_no_target, bins_no_target, value_range)
inv_cdf_with_target = extend_cdf_range(inv_cdf_with_target, bins_with_target, value_range)
plt.figure()
line, = plt.plot(inv_cdf_no_target, inv_cdf_with_target)
plt.xlim([-0.001,0.1001])
plt.ylim([-0.01,1])
plt.axline((0,0),(0.1,0.1),color=line.get_color(),linestyle="--")
plt.ylabel("P-TP")
plt.xlabel("P-FP")

def lin_interp(x1,y1,x2,y2,x):
    m = (y2-y1)/(x2-x1)
    n = y1
    return m*(x-x1) + n
def A_value(inv_cdf_no_target, inv_cdf_with_target, threshold):
    inv_cdf_no_target = inv_cdf_no_target[::-1]
    inv_cdf_with_target = inv_cdf_with_target[::-1]
    triangle_area = 0.5 * threshold**2
    cutoff = np.argmax(inv_cdf_no_target > threshold)-1
    cutoff = np.max([0,cutoff])
    
    point = lin_interp(
        inv_cdf_no_target[cutoff],
        inv_cdf_with_target[cutoff],
        inv_cdf_no_target[cutoff+1],
        inv_cdf_with_target[cutoff+1],
        threshold
    )
    inv_cdf_no_target = np.concatenate((inv_cdf_no_target[:cutoff], [threshold]))
    inv_cdf_with_target = np.concatenate((inv_cdf_with_target[:cutoff], [point]))
    
    area_under_curve = np.trapz(inv_cdf_with_target, inv_cdf_no_target, )
    
    # area_under_curve = point*threshold/2
    return (area_under_curve - triangle_area)/(threshold - triangle_area)
A_value(inv_cdf_no_target,inv_cdf_with_target,0.3), A_value(inv_cdf_no_target,inv_cdf_with_target,0.1),A_value(inv_cdf_no_target,inv_cdf_with_target,0.01), A_value(inv_cdf_no_target,inv_cdf_with_target,0.001)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.5615507030135164,
 0.298703661129998,
 0.022727917632725466,
 0.001157556217522073)

In [9]:
from matplotlib import pyplot as plt
bins = get_shared_bins(1000, clustered_est_no_target, clustered_est_with_target)
count_clustered_no_target,bins_clustered_no_target = np.histogram(clustered_est_no_target.reshape(-1,),bins=bins)
count_clustered_with_target,bins_clustered_with_target =np.histogram(clustered_est_with_target.reshape(-1,),bins=bins)
fig = plt.figure()
plt.plot(bins_centers(bins_clustered_no_target), count_clustered_no_target, zorder=1)
plt.plot(bins_centers(bins_clustered_with_target), count_clustered_with_target, zorder=3)
plt.plot(bins_centers(bins_with_target), count_with_target, c="k", ls="--", zorder=2)
plt.legend(["clustered-no-target", "clustered-with-target", "not-clustered-with-target"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure()
plt.plot(inv_cdf_no_target, inv_cdf_with_target, ls="--", c="k")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
from scipy.interpolate import pchip

def get_inv_cdf(count, bins):
    cdf = np.cumsum(count)/np.sum(count)
    return 1 - cdf/cdf[-1]

def extend_cdf_range(samples, sample_points, new_sample_points):
    out = np.zeros(len(new_sample_points))
    current_point_index = 1
    current_point = sample_points[current_point_index]
    current_sample = samples[current_point_index]
    for i, sample_point in enumerate(new_sample_points):
        if sample_point >= current_point:
            current_point_index += 1
        if current_point_index >= len(sample_points):
            current_sample = 0
        else: 
            current_sample = samples[current_point_index-1]
        out[i] = current_sample
    return out

inv_cdf_no_target = get_inv_cdf(count_clustered_no_target, bins_clustered_no_target)
inv_cdf_with_target = get_inv_cdf(count_clustered_with_target, bins_clustered_with_target)
value_range = np.sort(np.concatenate((bins_clustered_no_target,bins_clustered_with_target)))
inv_cdf_no_target = extend_cdf_range(inv_cdf_no_target, bins_clustered_no_target, value_range)
inv_cdf_with_target = extend_cdf_range(inv_cdf_with_target, bins_clustered_with_target, value_range)

line, = plt.plot(inv_cdf_no_target, inv_cdf_with_target)
plt.xlim([-0.001,0.1001])
plt.ylim([-0.01,1])
plt.axline((0,0),(0.1,0.1),color=line.get_color(),linestyle="--")
A_value(inv_cdf_no_target,inv_cdf_with_target,0.3), A_value(inv_cdf_no_target,inv_cdf_with_target,0.1),A_value(inv_cdf_no_target,inv_cdf_with_target,0.01), A_value(inv_cdf_no_target,inv_cdf_with_target,0.001)

(0.6798230905567975,
 0.48192924781503715,
 0.12356610473963439,
 0.016706094995839846)

In [12]:
plt.legend(["non-clustered","clustered", "diagonal reference"])

In [13]:
# M = neighbour_mean(img)
# norms = np.linalg.norm(img - M,axis=-1).reshape(-1,)
# targets = np.array([60*img.shape[1] + 70, norms.argmin(), norms.argmax(), 219*img.shape[1] + 187])
# # targets = np.random.choice(img.shape[0]*img.shape[1], size=5)
# targets = np.unravel_index(targets,img.shape[:2])
# targets = img[targets]
targets=np.zeros((7,126))
for i in range(7):
    targets[i] = img[clustered_image == i].mean(0)
targets.shape

(7, 126)

In [17]:
fig, axes = plt.subplots((len(targets)+1)//2,2)
fig = plt.figure()
roc_axes = fig.gca()
roc_colors = [None]*len(targets)
# t = img[60,70]
# no_target, with_target= original_algorithm(img, p, t)
# bins = get_shared_bins(1000, no_target, with_target)
# count_no_target,bins_no_target = np.histogram(no_target.reshape(-1,),bins=bins)
# count_with_target,bins_with_target =np.histogram(with_target.reshape(-1,),bins=bins)
# no_target_line, = axes[0,0].plot(bins_centers(bins_no_target), count_no_target,ls="--")
# with_target_line, = axes[0,0].plot(bins_centers(bins_with_target), count_with_target,ls="--")

# no_target_color= no_target_line.get_color()
# with_target_color= with_target_line.get_color()

# inv_cdf_no_target = get_inv_cdf(count_no_target, bins_no_target)
# inv_cdf_with_target = get_inv_cdf(count_with_target, bins_with_target)
# value_range = np.sort(np.concatenate((bins_no_target,bins_with_target)))
# inv_cdf_no_target = extend_cdf_range(inv_cdf_no_target, bins_no_target, value_range)
# inv_cdf_with_target = extend_cdf_range(inv_cdf_with_target, bins_with_target, value_range)
# roc_line, = roc_axes.plot(inv_cdf_no_target, inv_cdf_with_target, ls="--")
# roc_colors[0] = roc_line.get_color()

# print("A-GMF-0", A_value(inv_cdf_no_target,inv_cdf_with_target,0.3), A_value(inv_cdf_no_target,inv_cdf_with_target,0.1),A_value(inv_cdf_no_target,inv_cdf_with_target,0.01), A_value(inv_cdf_no_target,inv_cdf_with_target,0.001))

for i,t in enumerate(tqdm(targets)):    
    no_target, with_target = original_algorithm(img, p, t)
    bins = get_shared_bins(1000, no_target, with_target)
    count_with_target,bins_with_target =np.histogram(with_target.reshape(-1,),bins=bins)
    count_no_target,bins_no_target = np.histogram(no_target.reshape(-1,),bins=bins)
    ax = axes[(i)//2,(i)%2]
    temp, = ax.plot(bins_centers(bins_no_target), count_no_target,ls="--")
    no_target_color = temp.get_color()
    temp, = ax.plot(bins_centers(bins_with_target), count_with_target,ls="--")
    with_target_color = temp.get_color()
    ax.set_title("Target #%d" % (i+1))
    
    
    inv_cdf_no_target = get_inv_cdf(count_no_target, bins_no_target)
    inv_cdf_with_target = get_inv_cdf(count_with_target, bins_with_target)
    value_range = np.sort(np.concatenate((bins_no_target,bins_with_target)))
    inv_cdf_no_target = extend_cdf_range(inv_cdf_no_target, bins_no_target, value_range)
    inv_cdf_with_target = extend_cdf_range(inv_cdf_with_target, bins_with_target, value_range)

    roc_line, = roc_axes.plot(inv_cdf_no_target, inv_cdf_with_target, ls="--")
    roc_colors[i] = roc_line.get_color()
    print("A-GMF-%d" % (i+1), A_value(inv_cdf_no_target,inv_cdf_with_target,0.3), A_value(inv_cdf_no_target,inv_cdf_with_target,0.1),A_value(inv_cdf_no_target,inv_cdf_with_target,0.01), A_value(inv_cdf_no_target,inv_cdf_with_target,0.001))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/7 [00:00<?, ?it/s]

A-GMF-1 0.4114637272591318 0.1840842481696429 0.01296716427037264 0.00042644189601177157
A-GMF-2 0.208860866669289 0.08226183391396266 0.004383461672200277 0.00011572072707272242
A-GMF-3 0.49110458699779214 0.21830677297612075 0.011718470581542902 0.0006219328573725649
A-GMF-4 0.3355974168538281 0.14082053476345244 0.009809750835792842 0.0002542675594812774
A-GMF-5 0.740469262764124 0.4888989531783192 0.011634252456478783 0.0005666836128523422
A-GMF-6 0.3110224958204306 0.1262121185677593 0.008351434198765132 0.00020078256147024282
A-GMF-7 0.3372336040983206 0.13457607044062603 0.0075367739203051525 0.00015901576653851808


In [18]:
# t = img[60,70]
# no_target, with_target= clustered_algorithm(img, p, t, clustered_image)
# bins = get_shared_bins(1000, no_target, with_target)
# count_no_target,bins_no_target = np.histogram(no_target.reshape(-1,),bins=bins)
# count_with_target,bins_with_target =np.histogram(with_target.reshape(-1,),bins=bins)
# axes[0,0].plot(bins_centers(bins_no_target), count_no_target, c=no_target_color)
# axes[0,0].plot(bins_centers(bins_with_target), count_with_target, c=with_target_color)

# inv_cdf_no_target = get_inv_cdf(count_no_target, bins_no_target)
# inv_cdf_with_target = get_inv_cdf(count_with_target, bins_with_target)
# value_range = np.sort(np.concatenate((bins_no_target,bins_with_target)))
# inv_cdf_no_target = extend_cdf_range(inv_cdf_no_target, bins_no_target, value_range)
# inv_cdf_with_target = extend_cdf_range(inv_cdf_with_target, bins_with_target, value_range)
# roc_axes.plot(inv_cdf_no_target, inv_cdf_with_target, c=roc_colors[0])
# print("A-SMF-%d" % (0), A_value(inv_cdf_no_target,inv_cdf_with_target,0.3), A_value(inv_cdf_no_target,inv_cdf_with_target,0.1),A_value(inv_cdf_no_target,inv_cdf_with_target,0.01), A_value(inv_cdf_no_target,inv_cdf_with_target,0.001))

for i,t in enumerate(tqdm(targets)):
    no_target, with_target= clustered_algorithm(img, p, t, clustered_image)
    bins = get_shared_bins(1000, no_target, with_target)
    count_no_target,bins_no_target = np.histogram(no_target.reshape(-1,),bins=bins)
    count_with_target,bins_with_target =np.histogram(with_target.reshape(-1,),bins=bins)
    axes[(i)//2,(i)%2].plot(bins_centers(bins_no_target), count_no_target, c=no_target_color)
    axes[(i)//2,(i)%2].plot(bins_centers(bins_with_target), count_with_target, c=with_target_color)
    
    inv_cdf_no_target = get_inv_cdf(count_no_target, bins_no_target)
    inv_cdf_with_target = get_inv_cdf(count_with_target, bins_with_target)
    value_range = np.sort(np.concatenate((bins_no_target,bins_with_target)))
    inv_cdf_no_target = extend_cdf_range(inv_cdf_no_target, bins_no_target, value_range)
    inv_cdf_with_target = extend_cdf_range(inv_cdf_with_target, bins_with_target, value_range)
    roc_axes.plot(inv_cdf_no_target, inv_cdf_with_target, c=roc_colors[i])
    print("A-SMF-%d" % (i), A_value(inv_cdf_no_target,inv_cdf_with_target,0.3), A_value(inv_cdf_no_target,inv_cdf_with_target,0.1),A_value(inv_cdf_no_target,inv_cdf_with_target,0.01), A_value(inv_cdf_no_target,inv_cdf_with_target,0.001))

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

A-SMF-0 0.5975260944709283 0.4197713728291413 0.09796697260953668 0.011933967345476134


  0%|          | 0/7 [00:00<?, ?it/s]

A-SMF-1 0.40504618458893227 0.24167772401044518 0.03204669953722701 0.001468811061416291


...

A-SMF-4 0.7625850096960478 0.664155890288457 0.5343290884483941 0.40489518582101647


  0%|          | 0/7 [00:00<?, ?it/s]

A-SMF-5 0.5421229252825327 0.3761921938235064 0.0793590501134498 0.009780790476950756


  0%|          | 0/7 [00:00<?, ?it/s]

A-SMF-6 0.6205929535665651 0.4905647279920949 0.18263551881504564 0.01686036636781376


In [194]:
roc_axes.legend(["GMF-"+t for t in ["t%d"%(i+1) for i in range(len(targets))]] + ["SMF-"+t for t in ["t%d"%(i+1) for i in range(len(targets))]])

In [183]:
no_target, with_target = original_algorithm(img, p, targets[0])
bins = get_shared_bins(1000, no_target, with_target)
count_with_target,bins_with_target =np.histogram(with_target.reshape(-1,),bins=bins)
count_no_target,bins_no_target = np.histogram(no_target.reshape(-1,),bins=bins)

In [184]:
gmf_no_target = count_no_target

In [185]:
no_target, with_target= clustered_algorithm(img, p, targets[0], clustered_image)
bins = get_shared_bins(1000, no_target, with_target)
count_no_target,bins_no_target = np.histogram(no_target.reshape(-1,),bins=bins)
count_with_target,bins_with_target =np.histogram(with_target.reshape(-1,),bins=bins)


  0%|          | 0/7 [00:00<?, ?it/s]

In [205]:
smf_lines = """A-SMF-0 0.5975260944709283 0.4197713728291413 0.09796697260953668 0.011933967345476134
A-SMF-1 0.40504618458893227 0.24167772401044518 0.03204669953722701 0.001468811061416291
A-SMF-2 0.625668682898064 0.4830727077063503 0.15658763189299732 0.01437229097648318
A-SMF-3 0.5427044756596149 0.36574850911484713 0.07105068270241534 0.009000804321455617
A-SMF-4 0.7625850096960478 0.664155890288457 0.5343290884483941 0.40489518582101647
A-SMF-5 0.5421229252825327 0.3761921938235064 0.0793590501134498 0.009780790476950756
A-SMF-6 0.6205929535665651 0.4905647279920949 0.18263551881504564 0.01686036636781376""".split("\n")
gmf_lines = """A-GMF-1 0.4114637272591318 0.1840842481696429 0.01296716427037264 0.00042644189601177157
A-GMF-2 0.208860866669289 0.08226183391396266 0.004383461672200277 0.00011572072707272242
A-GMF-3 0.49110458699779214 0.21830677297612075 0.011718470581542902 0.0006219328573725649
A-GMF-4 0.3355974168538281 0.14082053476345244 0.009809750835792842 0.0002542675594812774
A-GMF-5 0.740469262764124 0.4888989531783192 0.011634252456478783 0.0005666836128523422
A-GMF-6 0.3110224958204306 0.1262121185677593 0.008351434198765132 0.00020078256147024282
A-GMF-7 0.3372336040983206 0.13457607044062603 0.0075367739203051525 0.00015901576653851808""".split("\n")

for smf, gmf in zip(smf_lines, gmf_lines):
    smf_entries = smf.split(" ")[1:]
    gmf_entries = gmf.split(" ")[1:]
    for smf_entry, gmf_entry in zip(smf_entries[::-1], gmf_entries[::-1]):
        print("%.3f" % (float(smf_entry)/float(gmf_entry)), end=", ")
    print()

27.985, 7.555, 2.280, 1.452, 
12.693, 7.311, 2.938, 1.939, 
23.109, 13.362, 2.213, 1.274, 
35.399, 7.243, 2.597, 1.617, 
714.500, 45.927, 1.358, 1.030, 
48.713, 9.502, 2.981, 1.743, 
106.030, 24.233, 3.645, 1.840, 


In [219]:
# Target examining

for i in range(len(targets)):
    cluster = img[clustered_image == i]
    print("Target %d" % (i+1), "||mean||", np.linalg.norm(np.mean(cluster,axis=0)), "||mean-median||", np.linalg.norm(np.mean(cluster,axis=0) - np.median(cluster,axis=0)), "||std||", np.linalg.norm(np.std((img-M)[clustered_image == i], axis=0)))


Target 1 ||mean|| 26066.128703934362 ||mean-median|| 186.2252271263063 ||std|| 884.9158966457151
Target 2 ||mean|| 9063.502487951404 ||mean-median|| 442.85833491671923 ||std|| 927.454254553328
Target 3 ||mean|| 28256.819274789872 ||mean-median|| 136.98004271296253 ||std|| 402.30601442025505
Target 4 ||mean|| 24363.367648086023 ||mean-median|| 521.9486292262637 ||std|| 1213.2113882349388
Target 5 ||mean|| 21630.080206886872 ||mean-median|| 440.4925086556825 ||std|| 575.3805342060458
Target 6 ||mean|| 24613.681777586688 ||mean-median|| 765.2549184995504 ||std|| 1297.2149167510922
Target 7 ||mean|| 26141.254294697166 ||mean-median|| 1499.31985035349 ||std|| 1798.7415259337938


In [19]:
data = img.copy()
M = neighbour_mean(data)
clusters_covariance = np.array([np.linalg.inv(covariance_matrix((data-M)[clustered_image == k][np.newaxis])) for k in tqdm(range(len(cluster_names)))])

  0%|          | 0/7 [00:00<?, ?it/s]

In [21]:
[np.all(np.linalg.eigvals(mat) > 0) for mat in clusters_covariance]

[True, True, True, True, True, True, True]